# AICO Demonstration

In this demonstration, we showcase the **Add-In Covariates (AICO)** test using three different datasets to illustrate its performance and usage:

1. **Simulated Regression Data**
2. **Simulated Classification Data**
3. **Empirical Data**




## Importing Necessary Libraries and Configuring Environment

To begin, we import the essential libraries and configure our development environment.

In [1]:
%load_ext autoreload
%autoreload 2

import os
import pandas as pd
import keras
from sklearn.model_selection import train_test_split

from src.simulation import sim_reg_model, sim_class_model
from src.score import neg_squared_loss, neg_binary_cross_entropy_loss
from src.aico import AICO

## Simulated Regression Data Example

In this section, we demonstrate the usage of the AICO test using a synthetic regression dataset. The target variable $Y$ is generated according to the following model:

$$
Y = \mu(X) + \epsilon
$$

where

$$
\mu(X) = 3 + 4X_1 + X_1X_2 + 3X_3^2 + 2X_4X_5 + 6X_6 + 2\sin(X_7) + \exp(X_8) + 5X_9 + 3X_{10} + 4X_{11} + 5X_{12}
$$

This equation combines linear and nonlinear relationships among the features, making it an ideal scenario to showcase the performance of AICO in evaluating feature significance.

**Significant Variables**

The following are the **Significant variables** used to generate the target variable $Y$:

- $(X_1, X_6) \sim N(0, \Sigma)$, with $\text{var}(X_1) = \text{var}(X_6) = 1$ and $\text{cov}(X_1, X_6) = 0.85$
- $X_2, X_3, X_4, X_5, X_7 \sim N(0, 1)$ independently
- $X_8 \sim \text{Uniform}[-1, 1]$
- $X_9 = 1\{X_2 + Z < 0\}$, where $Z \sim N(0, 1)$
- $X_{10} \sim \text{Poisson}(3)$
- $X_{11}, X_{12} \sim t(5)$ independently

**Insignificant Variables**

The **remaining variables** that do not directly influence $Y$ are defined as follows:

- $X_{13} \sim N(0, 1)$, with $\text{cov}(X_{12}, X_1) = \text{cov}(X_{12}, X_6) = 0.85$
- $X_{14} \sim N(0, 1)$ independently
- $(X_{15}, X_{16}) \sim N(0, \Sigma)$, with $\text{var}(X_i) = 1$ and $\text{cov}(X_{15}, X_{16}) = 0.85$
- $X_{17}, X_{18}, X_{19} \sim t(5)$ independently
- $\epsilon \sim N(0, 1)$


### Configuration

To ensure reproducibility, we start by setting a seed value. In this demonstration, we use an initial seed of 100. The trained models are also provided for seed = [100, 200, ..., 1000].

In [2]:
seed = 100
n_train, n_test = 1000000, 500000
score_func = neg_squared_loss

### Setup

In [3]:
model_dir = f'models/sim_reg'
data_sim = sim_reg_model

model = keras.models.load_model(os.path.join(model_dir, f'{seed}.keras'))
x_train, x_test, y_train, y_test = data_sim(n_train, n_test, seed)
x_train = pd.DataFrame(x_train, columns=[f'X{k}' for k in range(20)])
x_test = pd.DataFrame(x_test, columns=[f'X{k}' for k in range(20)])

/Users/cjiracho/anaconda3/envs/AICO/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adam', because it has 10 variables whereas the saved optimizer has 2 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


In [4]:
display(x_test)

,X0,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19
0,1.0,-0.616415,0.446946,0.936181,-1.644965,-1.658685,-1.585796,-1.974250,0.832280,0.0,6.0,-0.270686,0.786204,0.945587,0.087847,-1.967256,-3.135299,-0.784501,-1.318783,0.925733
1,1.0,-0.475787,0.781935,0.249289,-0.837013,1.017539,-0.268717,-1.030753,-0.600192,0.0,4.0,1.013083,-0.951465,0.457094,0.831777,0.566845,0.427276,1.024684,-1.060713,0.104843
2,1.0,0.037287,0.424161,0.535530,-0.176636,-0.012746,0.355387,-0.219245,-0.667496,0.0,5.0,0.461231,-0.569076,0.257177,0.740125,0.549627,0.904900,-0.948701,-0.296842,1.808831
3,1.0,1.462322,-0.526371,0.481497,-0.768913,2.195938,1.814091,0.421102,0.824298,1.0,2.0,0.230448,-3.618840,-1.669984,-0.777348,-1.026365,-2.151867,0.610035,0.249268,0.253275
4,1.0,-0.332206,1.027999,-0.424191,-0.905203,-1.466015,0.432213,-0.144542,-0.072044,0.0,1.0,-2.883468,-0.946597,0.710079,-1.361953,0.674581,1.042105,0.291124,1.154637,-0.562280
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,1.0,-0.264619,0.329488,0.712887,-1.009892,0.607698,-0.839730,-0.014784,0.445002,0.0,1.0,-0.571428,1.637662,-1.029180,-0.421150,1.052411,0.485048,-1.510390,-0.776908,-0.027357
499996,1.0,1.498497,0.487832,0.325633,-0.054356,-0.331191,1.232865,-0.211139,0.884082,0.0,3.0,-0.342413,-0.349674,0.800757,0.551736,0.543393,0.714731,0.932579,-1.508284,-1.261260
499997,1.0,1.164817,-0.804159,0.309731,0.612539,0.888757,1.319801,0.363912,-0.699816,1.0,6.0,-0.632495,-1.961283,0.005131,-1.715383,-0.960883,-0.975108,0.161922,0.991894,0.434644
499998,1.0,1.123457,0.683942,-0.425928,0.197902,-0.659447,1.351228,-0.534417,-0.757448,0.0,3.0,0.431309,-0.524256,-0.004897,-0.059330,-0.389469,-0.141231,1.198804,1.414448,1.392029


In [5]:
display(y_test)

array([19.11207353,  6.70627446, 20.47036031, ..., 28.88143421,
       25.70374177, 10.06703483])

### AICO Test

In this section, we initialize and run the Add-In Covariates (AICO) test to evaluate feature significance.

1. **Initializing AICO**:
   - We create an instance of the `AICO` class using the training data and model predictions.
   - **Parameters**:
     - `x_train`, `y_train`: Training features and target variable used to build the baseline features.
     - `pred_func`: The prediction function of the model (`model.predict`) is used to obtain predictions from the trained model.
     - `pred_params`: Parameters for the prediction function, such as `batch_size` and `verbose` level. Here, we use the test dataset size as the batch size and set verbosity to 0 for a silent run.
     - `score_func`: The scoring function to evaluate model performance. This measures the change in prediction accuracy when adding each feature.
     - `ignored_vars`: Variables that should be ignored during the AICO test (intercept `['X0']` in this example).
     - `discrete_vars`: Variables that are treated as discrete (`['X10']`).
     - `categorical_vars`: Variables treated as categorical (`['X9']`). More details are explained in the empirical data section.

2. **Running the AICO Test**:
   - The `aico.test` function is called using the test data (`x_test` and `y_test`), which allows us to evaluate the significance of each feature by comparing the model's performance with and without that feature.

3. **Displaying the AICO Summary**:
   - Finally, we call `aico.summary()` to display the results of the test. The summary provides insights into which features have a statistically significant impact on the model's performance, helping identify the most influential covariates.


In [6]:
aico = AICO(x_train=x_train,
            y_train=y_train, 
            pred_func=model.predict,
            pred_params=dict(batch_size=y_test.size, verbose=0),
            score_func=score_func,
            ignored_vars=['X0'],
            discrete_vars=['X10'],
            categorical_vars=['X9'])
aico.test(x_test=x_test,
          y_test=y_test)
aico.summary()

================================[AICO Test Result Summary]==================================
alpha:              0.05                               # testing samples:       500,000
# tested variables: 19                                 # significant variables: 12
score function:     neg_squared_loss                   
[Significant Variables]
rank   variable             median       p_value      [lower       upper]       coverage    
--------------------------------------------------------------------------------------------
1      X9                   24.993       0.000        24.957       25.029       0.95018     
2      X6                   15.788       0.000        15.683       15.902       0.95018     
3      X10                  14.572       0.000        14.521       14.623       0.95018     
4      X12                  12.898       0.000        12.805       12.995       0.95018     
5      X11                  8.137        0.000        8.075        8.200        0.95018     
6    

## Simulated Classification Data Example

In this section, we demonstrate the usage of the **Add-In Covariates (AICO)** test using a synthetic classification dataset. The target variable $Y$ is generated as a binary outcome following a logistic model, where:

$$
P(Y = 1 | X) = L(\mu(X))
$$

and

$$
L(u) = \frac{1}{1 + e^{-u}}
$$

The underlying function $\mu(X)$ and distributions of $X's$ are the same as in the regression example.

### Configuration

In [7]:
seed = 100
n_train, n_test = 1000000, 500000

### Setup

In [8]:
model_dir = f'models/sim_class'
data_sim = sim_class_model
score_func = neg_binary_cross_entropy_loss

model = keras.models.load_model(os.path.join(model_dir, f'{seed}.keras'))
x_train, x_test, y_train, y_test = data_sim(n_train, n_test, seed)
x_train = pd.DataFrame(x_train, columns=[f'X{k}' for k in range(20)])
x_test = pd.DataFrame(x_test, columns=[f'X{k}' for k in range(20)])

/Users/cjiracho/anaconda3/envs/AICO/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adam', because it has 10 variables whereas the saved optimizer has 2 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


In [9]:
display(x_test)

,X0,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19
0,1.0,-1.100663,1.745703,-1.245385,0.229655,-1.216234,-1.524658,-0.300961,-0.773008,0.0,2.0,-1.723812,0.485132,-0.671060,-1.158567,-0.309976,-0.350257,1.837634,1.366944,0.345471
1,1.0,-1.120371,-0.979782,0.140979,-1.116640,2.171029,-1.187076,-0.569585,0.688934,1.0,6.0,0.068478,0.998485,-0.636367,1.538031,0.043029,1.466982,1.516406,-2.057503,-0.065398
2,1.0,0.086887,-0.341548,-0.018576,0.220847,1.375975,0.481130,-0.052092,-0.200230,1.0,2.0,0.597324,0.209467,0.114707,0.353339,-0.819427,-0.543975,2.386803,-0.707401,0.620556
3,1.0,0.113405,-1.785407,-0.871968,0.807782,0.417833,0.590651,1.000630,-0.513311,1.0,1.0,-0.513507,-0.367501,-1.040576,0.116821,0.353130,0.175088,-0.490095,0.504334,-0.329850
4,1.0,-0.103607,0.928274,1.454881,-0.595645,-0.841883,-0.373759,0.262119,0.441101,0.0,1.0,1.389142,0.400297,0.468790,-1.878837,0.125549,1.313551,-4.804187,-0.695228,1.882249
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,1.0,-0.931714,1.062577,-0.227822,-0.970179,-0.810640,-0.442725,0.545558,0.618013,0.0,4.0,-0.408491,-1.641763,-0.796884,0.671904,0.112491,0.555100,-1.188166,-0.714091,2.133041
499996,1.0,-1.385707,0.491881,-0.501278,-1.341781,0.102413,0.029906,0.756042,-0.960020,1.0,4.0,-0.068591,0.744552,-0.121144,-0.962527,-1.142154,-1.041702,1.872096,-1.309278,-0.919981
499997,1.0,0.557040,-0.788253,-0.429355,0.686031,-0.509051,1.409448,-1.146241,-0.737663,1.0,3.0,-0.317596,0.442960,-0.376204,0.132390,-0.749140,0.488139,1.006802,1.340289,-1.245268
499998,1.0,-1.409004,-0.625428,1.376841,-1.899479,1.155096,-0.394647,0.087273,0.650275,0.0,1.0,0.273701,0.298279,-1.498264,1.077641,1.147289,1.043172,0.993436,-0.091074,0.114439


In [10]:
display(y_test)

array([1, 0, 0, ..., 0, 0, 0])

### AICO Test

In [11]:
aico = AICO(x_train=x_train,
            y_train=y_train, 
            pred_func=model.predict,
            pred_params=dict(batch_size=y_test.size, verbose=0),
            score_func=score_func,
            ignored_vars=['X0'],
            discrete_vars=['X10'],
            categorical_vars=['X9'])
aico.test(x_test=x_test,
          y_test=y_test)
aico.summary()

================================[AICO Test Result Summary]==================================
alpha:              0.05                               # testing samples:       500,000
# tested variables: 19                                 # significant variables: 12
score function:     neg_binary_cross_entropy_loss      
[Significant Variables]
rank   variable             median       p_value      [lower       upper]       coverage    
--------------------------------------------------------------------------------------------
1      X3                   3.577e-07    0.000        3.372e-07    3.770e-07    0.95018     
2      X10                  2.324e-08    0.000        2.113e-08    2.545e-08    0.95018     
3      X6                   6.722e-10    0.000        6.117e-10    7.425e-10    0.95018     
4      X1                   1.630e-10    0.000        1.469e-10    1.802e-10    0.95018     
5      X12                  6.989e-11    0.000        5.979e-11    8.150e-11    0.95018     
6    

## Empirical Data

In this section, we demonstrate the usage of the **Add-In Covariates (AICO)** test on a real-world dataset: the **Default of Credit Card Clients** dataset from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/350/default+of+credit+card+clients). This dataset contains information about credit card clients in Taiwan, including demographic information and payment history.

### Setup

In [12]:
seed = 100
x, y = pd.read_pickle('data/credit_default/x.pkl'), pd.read_pickle('data/credit_default/y.pkl')
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=1/4, random_state=seed, stratify=y)
x_train, x_val, y_train, y_val =  train_test_split(x_train, y_train, test_size=1/5, random_state=seed, stratify=y_train)

model = keras.models.load_model(f'models/credit_default/{seed}.keras')
pred_params = dict(batch_size=y_test.size, verbose=0)

/Users/cjiracho/anaconda3/envs/AICO/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adam', because it has 14 variables whereas the saved optimizer has 2 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


In [13]:
display(x_test)

,intercept,LIMIT_BAL,AGE,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,...,PAY_5_payment_delay_for_three_month,PAY_5_payment_delay_for_two_month,PAY_6_pay_duly,PAY_6_payment_delay_for_eight_month,PAY_6_payment_delay_for_five_month,PAY_6_payment_delay_for_four_month,PAY_6_payment_delay_for_seven_month,PAY_6_payment_delay_for_six_month,PAY_6_payment_delay_for_three_month,PAY_6_payment_delay_for_two_month
6205,1,0.176956,-0.811478,-0.693030,-0.689387,-0.682339,-0.672213,-0.664030,-0.652551,-0.336019,...,0,0,1,0,0,0,0,0,0,0
12102,1,-0.904734,0.599647,-0.044340,-0.023164,0.003282,0.017671,-0.210601,-0.187635,-0.219634,...,0,0,1,0,0,0,0,0,0,0
22204,1,-0.363889,-0.051642,0.281357,0.321530,0.294830,0.395415,0.480608,-0.604593,-0.149803,...,0,0,1,0,0,0,0,0,0,0
21501,1,0.331483,-0.377286,-0.670623,-0.675355,-0.682339,-0.644830,-0.652898,-0.639919,-0.277826,...,0,0,1,0,0,0,0,0,0,0
4980,1,-0.209362,-0.485834,1.001643,1.093254,1.189242,1.347183,1.521651,1.618549,0.001498,...,0,1,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16158,1,0.176956,-0.268738,-0.690162,-0.632826,-0.594411,-0.606314,-0.616091,-0.600108,-0.101445,...,0,0,1,0,0,0,0,0,0,0
21130,1,-0.904734,-1.028574,-0.037790,0.006975,0.029532,0.100099,0.156879,0.187010,-0.186406,...,0,0,1,0,0,0,0,0,0,0
8202,1,-0.518416,-0.702930,0.488795,0.689137,0.714424,0.782100,0.925627,0.964386,0.403026,...,0,1,0,0,0,0,0,0,0,1
383,1,0.331483,-0.377286,0.222465,0.285638,0.351016,0.443634,0.548121,0.604389,-0.149803,...,0,0,1,0,0,0,0,0,0,0


In [14]:
display(y_test)

6205     1
12102    0
22204    0
21501    0
4980     1
        ..
16158    0
21130    0
8202     1
383      0
19268    0
Name: default payment next month, Length: 7500, dtype: int64

This part of the demonstration specifically aims to show how to use the `categorical_vars` argument in AICO for handling categorical features:

- The `categorical_vars` argument can be provided as either a **dictionary** or a **list**.
  - **Dictionary Format**: Maps from the categorical variable's name to the list of corresponding dummy variables. For example:
  
    ```python
    categorical_vars = {
        'MARRIAGE': ['MARRIAGE_married', 'MARRIAGE_others', 'MARRIAGE_single'],
        'EDUCATION': ['EDUCATION_graduate_school', 'EDUCATION_high_school', 'EDUCATION_others', 'EDUCATION_university']
    }
    ```

  - **List Format**: If a list is provided, AICO will automatically construct such a dictionary by using each element as a prefix to match with the corresponding dummy variables. For instance, providing `['MARRIAGE', 'EDUCATION']` will automatically generate the dictionary shown above.

### AICO Test

In [15]:
aico = AICO(x_train=x_train,
            y_train=y_train, 
            pred_func=model.predict,
            pred_params=pred_params,
            score_func=neg_binary_cross_entropy_loss,
            alpha=0.05,
            ignored_vars=['intercept'],
            categorical_vars=['SEX', 'EDUCATION', 'MARRIAGE', 'PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6'])
aico.test(x_test=x_test, y_test=y_test)
aico.summary()

================================[AICO Test Result Summary]==================================
alpha:              0.05                               # testing samples:       7,500
# tested variables: 23                                 # significant variables: 7
score function:     neg_binary_cross_entropy_loss      
[Significant Variables]
rank   variable             median       p_value      [lower       upper]       coverage    
--------------------------------------------------------------------------------------------
1      PAY_2                0.187        0.000        0.183        0.192        0.95169     
2      PAY_3                0.174        0.000        0.170        0.178        0.95169     
3      PAY_4                0.154        0.000        0.149        0.157        0.95169     
4      PAY_0                0.143        0.000        0.139        0.146        0.95169     
5      PAY_5                0.115        0.000        0.113        0.117        0.95169     
6      P